In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from datetime import datetime, timedelta, timezone
from sklearn.model_selection import train_test_split

In [3]:
failure_2016 = pd.read_csv('../data/init/failures-2016.csv',sep=';')
failure_2017 = pd.read_csv('../data/init/failures-2017.csv',sep=';')
metmast_2016 = pd.read_csv('../data/init/metmast-2016.csv',sep=';')
metmast_2017 = pd.read_csv('../data/init/metmast-2017.csv',sep=';')
signals_2016 = pd.read_csv('../data/init/signals-2016.csv', sep=';')
signals_2017= pd.read_csv('../data/init/signals-2017.csv', sep=';')



# 1. Cleaning Signal data

Combining signals from both years and aggregating the time series data into recurring once a day.

In [4]:
def signal_preprocess(signals):
    
    signals['Timestamp'] = pd.to_datetime(signals['Timestamp'])
    signals=signals.set_index('Timestamp')
    return signals


In [5]:
signals=pd.concat([signals_2016, signals_2017], axis=0)
signals = signal_preprocess(signals)

Drop the columns with low variance

In [12]:
def get_signals_with_low_variance(df: pd.DataFrame, threshold=0) -> list:
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    cont_data = df.select_dtypes(include=numerics)
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(cont_data)
    inverted_list = ~np.array(selector.get_support())
    return cont_data.columns[inverted_list].tolist()

In [13]:
cols_to_drop=get_signals_with_low_variance(signals)
signals.drop(cols_to_drop, axis=1, inplace=True)

Aggregate the time series dataframe into a daily data frame

In [26]:
signals_aggregation_rules = {
    'Gen_RPM_Max': 'max',
    'Gen_RPM_Min': 'min',
    'Gen_RPM_Avg': 'mean',
    'Gen_RPM_Std': 'mean',
    'Gen_Bear_Temp_Avg': 'mean',
    'Gen_Phase1_Temp_Avg': 'mean',
    'Gen_Phase2_Temp_Avg': 'mean',
    'Gen_Phase3_Temp_Avg': 'mean',
    'Hyd_Oil_Temp_Avg': 'mean',
    'Gear_Oil_Temp_Avg': 'mean',
    'Gear_Bear_Temp_Avg': 'mean',
    'Nac_Temp_Avg': 'mean',
    'Rtr_RPM_Max': 'max',
    'Rtr_RPM_Min': 'min',
    'Rtr_RPM_Avg': 'mean',
    'Amb_WindSpeed_Max': 'max',
    'Amb_WindSpeed_Min': 'min',
    'Amb_WindSpeed_Avg': 'mean',
    'Amb_WindSpeed_Std': 'mean',
    'Amb_WindDir_Relative_Avg': 'mean',
    'Amb_WindDir_Abs_Avg': 'mean',
    'Amb_Temp_Avg': 'mean',
    'Prod_LatestAvg_ActPwrGen0': 'mean',
    'Prod_LatestAvg_ActPwrGen1': 'mean',
    #'Prod_LatestAvg_ActPwrGen2': 'mean',
    'Prod_LatestAvg_TotActPwr': 'sum',
    'Prod_LatestAvg_ReactPwrGen0': 'mean',
    'Prod_LatestAvg_TotReactPwr': 'sum',
    'HVTrafo_Phase1_Temp_Avg': 'mean',
    'HVTrafo_Phase2_Temp_Avg': 'mean',
    'HVTrafo_Phase3_Temp_Avg': 'mean',
    'Grd_InverterPhase1_Temp_Avg': 'mean',
    'Cont_Top_Temp_Avg': 'mean',
    'Cont_Hub_Temp_Avg': 'mean',
    'Cont_VCP_Temp_Avg': 'mean',
    'Gen_SlipRing_Temp_Avg': 'mean',
    'Spin_Temp_Avg': 'mean',
    'Blds_PitchAngle_Min': 'min',
    'Blds_PitchAngle_Max': 'max',
    'Blds_PitchAngle_Avg': 'mean',
    'Blds_PitchAngle_Std': 'mean',
    'Cont_VCP_ChokcoilTemp_Avg': 'mean',
    'Grd_RtrInvPhase1_Temp_Avg': 'mean',
    'Grd_RtrInvPhase2_Temp_Avg': 'mean',
    'Grd_RtrInvPhase3_Temp_Avg': 'mean',
    'Cont_VCP_WtrTemp_Avg': 'mean',
    'Grd_Prod_Pwr_Avg': 'mean',
    'Grd_Prod_CosPhi_Avg': 'mean',
    'Grd_Prod_Freq_Avg': 'mean',
    'Grd_Prod_VoltPhse1_Avg': 'mean',
    'Grd_Prod_VoltPhse2_Avg': 'mean',
    'Grd_Prod_VoltPhse3_Avg': 'mean',
    'Grd_Prod_CurPhse1_Avg': 'mean',
    'Grd_Prod_CurPhse2_Avg': 'mean',
    'Grd_Prod_CurPhse3_Avg': 'mean',
    'Grd_Prod_Pwr_Max': 'max',
    'Grd_Prod_Pwr_Min': 'min',
    'Grd_Busbar_Temp_Avg': 'mean',
    'Rtr_RPM_Std': 'mean',
    'Amb_WindSpeed_Est_Avg': 'mean',
    'Grd_Prod_Pwr_Std': 'mean',
    'Grd_Prod_ReactPwr_Avg': 'mean',
    'Grd_Prod_ReactPwr_Max': 'max',
    'Grd_Prod_ReactPwr_Min': 'min',
    'Grd_Prod_ReactPwr_Std': 'mean',
    'Grd_Prod_PsblePwr_Avg': 'mean',
    'Grd_Prod_PsblePwr_Max': 'max',
    'Grd_Prod_PsblePwr_Min': 'min',
    'Grd_Prod_PsblePwr_Std': 'mean',
    'Grd_Prod_PsbleInd_Avg': 'mean',
    'Grd_Prod_PsbleInd_Max': 'max',
    'Grd_Prod_PsbleInd_Min': 'min',
    'Grd_Prod_PsbleInd_Std': 'mean',
    'Grd_Prod_PsbleCap_Avg': 'mean',
    'Grd_Prod_PsbleCap_Max': 'max',
    'Grd_Prod_PsbleCap_Min': 'min',
    'Grd_Prod_PsbleCap_Std': 'mean',
    'Gen_Bear2_Temp_Avg': 'mean',
    'Nac_Direction_Avg': 'mean'
}

In [27]:
def aggregate_signals(signals):
    agg_signals=signals.groupby('Turbine_ID').resample('D').agg(signals_aggregation_rules)
    agg_signals['Turbine_ID'] = agg_signals.index.get_level_values('Turbine_ID')  
    agg_signals=agg_signals.reset_index('Timestamp')
    return agg_signals


In [28]:
agg_signals=aggregate_signals(signals)
agg_signals=agg_signals.reset_index(drop=True)

In [29]:
# binary column to indicate if a signal data has any missing values
agg_signals['missing_values'] = agg_signals.isnull().any(axis=1).astype(int)
#fill the missing valeu with backfill and forwardfill
signals_clean=agg_signals.fillna(method='bfill')
#signals_clean=signals_clean.reset_index()

/var/folders/7b/1zsp53s91z97clkc70s5xcym0000gn/T/ipykernel_28235/1721483605.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  signals_clean=agg_signals.fillna(method='bfill')


# 2. Clean Metacast data

Combine metamast data for both 2016 and 2017. Aggregate the metamast data into daily time slots

Metmast data do not have data from 2017-01-04 to 2017-05-05

In [31]:
metmast = pd.concat([metmast_2016, metmast_2017], axis=0)
metmast['Timestamp'] = pd.to_datetime(metmast['Timestamp'])
metmast=metmast.set_index('Timestamp')


Drop the columns with low variance

In [32]:
cols_to_drop = get_signals_with_low_variance(metmast)  
metmast = metmast.drop(cols_to_drop, axis=1)
metmast= metmast.drop(["Min_Winddirection2", "Max_Winddirection2", "Avg_Winddirection2", "Var_Winddirection2"], axis=1)

Aggregate the data according to the columns (with mean or sum)

In [33]:
aggregation_rules = {
    'Min_Windspeed1': 'min',
    'Max_Windspeed1': 'max',
    'Avg_Windspeed1': 'mean',
    'Var_Windspeed1': 'mean',
    'Min_Windspeed2': 'min',
    'Max_Windspeed2': 'max',
    'Avg_Windspeed2': 'mean',
    'Var_Windspeed2': 'mean',
    'Min_AmbientTemp': 'min',
    'Max_AmbientTemp': 'max',
    'Avg_AmbientTemp': 'mean',
    'Min_Pressure': 'min',
    'Max_Pressure': 'max',
    'Avg_Pressure': 'mean',
    'Min_Humidity': 'min',
    'Max_Humidity': 'max',
    'Avg_Humidity': 'mean',
    'Min_Precipitation': 'min',
    'Max_Precipitation': 'max',
    'Avg_Precipitation': 'mean',
    'Max_Raindetection': 'max',
    'Anemometer1_Avg_Freq': 'mean',
    'Anemometer2_Avg_Freq': 'mean',
    'Pressure_Avg_Freq': 'mean',
}


In [34]:
agg_metmast=metmast.resample('D').agg(aggregation_rules)
agg_metmast=agg_metmast.reset_index()


In [35]:
agg_metmast['metamast_missing_values'] = agg_metmast.isnull().any(axis=1).astype(int)
metmast_clean=agg_metmast.fillna(method='bfill')

/var/folders/7b/1zsp53s91z97clkc70s5xcym0000gn/T/ipykernel_28235/2464392033.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  metmast_clean=agg_metmast.fillna(method='bfill')


# Merge metamast data and signals data

In [36]:
def merge_signals_metmast(signals: pd.DataFrame, metmast: pd.DataFrame) -> pd.DataFrame:
    merged_df = pd.merge(signals, metmast, on="Timestamp", how="left")
    #merged_df.drop(columns=["index"], inplace=True)
    merged_df = merged_df[merged_df["missing_values"] == 0]
    merged_df = merged_df[merged_df["metamast_missing_values"] == 0 ]
    merged_df.drop(columns=["missing_values", "metamast_missing_values"], inplace=True)
    return merged_df


In [109]:
merged_df = merge_signals_metmast(signals_clean, metmast_clean)

In [108]:
# load the column description excel file
column_description = pd.read_excel('../data/init/column description.xlsx')

In [110]:
# change the expanded column names to the camel caps
column_description['Expanded_column_names'] = column_description['Expanded_column_names'].str.replace(' ', '_').str.lower()

In [111]:
# replace the merged_df column names with the expanded column names from column descriptio df except for the timestamp column and turbine id
merged_df.columns = column_description.set_index('Columns').loc[merged_df.columns, 'Expanded_column_names'].values

# 3. Clean Failure data

In [119]:
failures=pd.concat([failure_2016, failure_2017], axis=0)
failures['Timestamp'] = pd.to_datetime(failures['Timestamp'])
failures['Timestamp'] = failures['Timestamp'].dt.floor('d')
# change the column name Timestamp to timestamp and Turbine_ID to turbine_id   
failures.columns = ['turbine_id', 'component', 'timestamp','remarks']



In [120]:
failures

,turbine_id,component,timestamp,remarks
0,T01,GEARBOX,2016-07-18 00:00:00+00:00,Gearbox pump damaged
1,T06,GENERATOR,2016-07-11 00:00:00+00:00,Generator replaced
2,T06,GENERATOR,2016-07-24 00:00:00+00:00,Generator temperature sensor failure
3,T06,GENERATOR,2016-09-04 00:00:00+00:00,High temperature generator error
4,T06,GENERATOR,2016-10-27 00:00:00+00:00,Generator replaced
5,T06,GENERATOR,2016-10-02 00:00:00+00:00,Refrigeration system and temperature sensors i...
6,T06,HYDRAULIC_GROUP,2016-04-04 00:00:00+00:00,Error in pitch regulation
7,T07,GENERATOR_BEARING,2016-04-30 00:00:00+00:00,High temperature in generator bearing (replace...
8,T07,TRANSFORMER,2016-07-10 00:00:00+00:00,High temperature transformer
9,T07,TRANSFORMER,2016-08-23 00:00:00+00:00,High temperature transformer. Transformer refr...


In [124]:
days_lookback = 60
def create_failure_list(failures: pd.DataFrame, days_lookback: int, value_function, target_name: str = "Target") -> pd.DataFrame:
    failure_list = []

    for i in range(len(failures)):
        turbine_id = str(failures.iloc[i]["turbine_id"])
        failure_datetime = failures.iloc[i]["timestamp"]
        components = failures.iloc[i]["component"]
        rounded_datetime = failure_datetime.replace(hour=0, minute=0, second=0, microsecond=0)  # Round to the start of the day

        for j in range(days_lookback):
            delta = timedelta(days=j)
            new_datetime = rounded_datetime - delta
            datetime_formated = new_datetime.replace(tzinfo=timezone.utc)
            # Calculate the target value using the provided value_function
            target_value = value_function(j, days_lookback)
            failure_list.append([turbine_id, datetime_formated.isoformat(), components,target_value])
    
    failure_df = pd.DataFrame(failure_list, columns=["turbine_id", "timestamp", "component",target_name])
    return failure_df

In [130]:

class_target_name = "target_class"
regression_function = lambda i, j: i
classif_function = lambda i, j: 1

In [131]:
components = failures["component"].unique()
for component in components:
    
    globals()[f"failure_df_{component}"] = create_failure_list(failures[failures["component"] == component], days_lookback, classif_function, class_target_name)
    globals()[f"failure_df_{component}"]['timestamp'] = pd.to_datetime(globals()[f"failure_df_{component}"]['timestamp'])
    globals()[f"labeled_df_{component}"] = pd.merge(merged_df.reset_index(drop=True), globals()[f"failure_df_{component}"].reset_index(drop=True), on=["turbine_id", "timestamp"], how="left")
    globals()[f"labeled_df_{component}"][class_target_name] = globals()[f"labeled_df_{component}"][class_target_name].fillna(0).astype(int)
    globals()[f"labeled_df_{component}"].drop_duplicates(inplace=True)
    


In [133]:
import os
os.makedirs('../data/model_data', exist_ok=True)

In [134]:
for component in components:
    globals()[f"labeled_df_{component}"].to_csv(f'../data/model_data/labelled_data_{component}.csv', index=False)


In [135]:
failures.to_csv('../data/model_data/failures.csv', index=False)